<a href="https://colab.research.google.com/github/MaxiPublica/maxibandas-api-pictures/blob/master/Soluci%C3%B3n_a_precios_at%C3%ADpicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Propuesta de soución a precios atípicos.
Se parte del los resultados del paso previo de "Análisis de precios atípicos"

## Preparación de los datos

### conecta con drive

In [1]:
# conecta con drive y carga librerías
from google.colab import drive
import shutil
import pandas as pd
import numpy as np
import os
from datetime import datetime, timezone, timedelta
from sklearn.linear_model import LinearRegression

# Definir el huso horario de la Ciudad de México (GMT-6/GMT-5)
huso_horario_mexico = timezone(timedelta(hours=-6))  # Durante horario estándar

# Monta tu Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### copia los archivos de datos de drive a local

In [2]:

# Ruta del directorio a copiar desde la unidad de drive a Colab
ruta_drive_datos = '/content/drive/My Drive/Colab_Notebooks/precios/datos/'

# Ruta del directorio a copiar desde la unidad de drive a Colab
ruta_drive_datos_tmp = '/content/drive/My Drive/Colab_Notebooks/precios/tmp/'

# Ruta de la carpeta local 'datos'
ruta_local_datos = '/content/datos/'
ruta_local_datos_temporal = ruta_local_datos +'tmp'


# Verificar si la carpeta 'datos' ya existe
if not os.path.exists(ruta_local_datos):
    # Si no existe, crearla
    os.makedirs(ruta_local_datos)
    print("Se ha creado la carpeta 'datos'")
else:
    print("La carpeta 'datos' ya existe")

# Verificar si la carpeta 'datos/temp' ya existe
if not os.path.exists(ruta_local_datos_temporal):
    # Si no existe, crearla
    os.makedirs(ruta_local_datos_temporal)
    print("Se ha creado la carpeta 'datos/temp'")
else:
    print("La carpeta 'datos/temp' ya existe")

# Obtener la lista de archivos en la carpeta de origen (ruta_drive_datos)
archivos = os.listdir(ruta_drive_datos)

# Copiar cada archivo de la carpeta 'datos' de origen a la carpeta local de destino
for archivo in archivos:
    ruta_origen = os.path.join(ruta_drive_datos, archivo)
    ruta_destino = os.path.join(ruta_local_datos, archivo)

    # Verificar si el archivo ya existe en la carpeta de destino
    if not os.path.exists(ruta_destino):
        shutil.copy2(ruta_origen, ruta_destino)
        print(f"Se ha copiado el archivo '{archivo}' correctamente")
    else:
        print(f"El archivo '{archivo}' ya existe en la carpeta local 'datos'")

# Obtener la lista de archivos en la carpeta 'tmp' de origen (ruta_drive_datos)
archivos = os.listdir(ruta_drive_datos_tmp)

# Copiar cada archivo de la carpeta 'datos' de origen a la carpeta local de destino
for archivo in archivos:
    ruta_origen = os.path.join(ruta_drive_datos_tmp, archivo)
    ruta_destino = os.path.join(ruta_local_datos_temporal, archivo)

    # Verificar si el archivo ya existe en la carpeta de destino
    if not os.path.exists(ruta_destino):
        shutil.copy2(ruta_origen, ruta_destino)
        print(f"Se ha copiado el archivo '{archivo}' correctamente")
    else:
        print(f"El archivo '{archivo}' ya existe en la carpeta local 'datos'")

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


La carpeta 'datos' ya existe
La carpeta 'datos/temp' ya existe
El archivo 'Repore_precios_ebc_gam_vs_mxp_update_2023_11_24_all_3_meses_antes_oct_nov.xlsx' ya existe en la carpeta local 'datos'
El archivo 'fecha_to_periodo.csv' ya existe en la carpeta local 'datos'
El archivo 'Repore_precios_ebc_gam_vs_mxp_update_2023_11_24_all_3_meses_antes_oct_nov_original.xlsx' ya existe en la carpeta local 'datos'
El archivo 'Repore_precios_ebc_gam_vs_mxp_update_2023_12_06_Noviembre_2023_historico_todas las versiones.xlsx' ya existe en la carpeta local 'datos'
El archivo 'Reporte_precios_ebc_gam_vs_mxp_update_2023_12_11_Noviembre_2023_historico_todas las versiones.xlsx' ya existe en la carpeta local 'datos'
El archivo '01_resumen_datos_historicos.csv' ya existe en la carpeta local 'datos'
El archivo '08_atípico_mm_ajustados_vs_libros.xlsx' ya existe en la carpeta local 'datos'
El archivo '04_cuantiles_ponderados_brand_model_year.csv' ya existe en la carpeta local 'datos'
El archivo '09_EBC_mma_nega

### Lee los datos desde los archivos de Excel y CSV y los carga en dataframes

In [3]:
# Ruta del archivo CSV
archivo_datos_historicos = 'datos/tmp/01_datos_historicos.csv'
archivo_mma_cuantiles_ponderados  = 'datos/tmp/06_brand_model_year_diferencia_ponderada_vs_libros.csv'

# Leer el archivo CSV y cargar los datos en un DataFrame
df  = pd.read_csv(archivo_datos_historicos)
df_mma = pd.read_csv(archivo_mma_cuantiles_ponderados)

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


Terminé: 2023-12-12 20:52:56 UTC-06:00


#### Aux. para encontrar las columnas diferentes entre ambos df

In [4]:
df
df.columns

Index(['Fecha', 'categoryId', 'brand', 'model', 'year', 'trim', 'MaxiPublica',
       'muestras por mes', 'muestra total últimos meses', 'muestra total',
       'emulado con versión anterior', 'emulado con EBC',
       'emulado por versiones hermanas', 'priceSaleEbc', 'pricePurchaseEbc',
       'priceSaleGam', 'pricePurchaseGam', 'EbcId', 'AutometricaId',
       'llave_mma', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por versión', 'MP_vs_EBC',
       'MP_vs_ATM'],
      dtype='object')

In [5]:
# Aux. para encontrar las columnas diferentes entre ambos df
lista_mm = ['Fecha', 'categoryId', 'brand', 'model', 'year', 'trim', 'MaxiPublica', 'muestras por mes', 'muestra total últimos meses', 'muestra total', 'emulado con versión anterior', 'emulado con EBC', 'emulado por versiones hermanas', 'priceSaleEbc', 'pricePurchaseEbc', 'priceSaleGam', 'pricePurchaseGam', 'EbcId', 'AutometricaId', 'llave_mma', 'Cantidad de versiones por año', 'Suma de muestra total últimos meses', 'Autos por versión', 'MP_vs_EBC', 'MP_vs_ATM']

lista_mma = ['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC', 'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM', 'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM', 'límite superior ATM', 'Q4_ATM', 'total elementos', 'EBC con outlier inferior', 'EBC con outlier superior', 'ATM con outlier inferior', 'ATM con outlier superior', '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior', 'llave_mma', 'llave_mm', 'Cantidad de versiones por año', 'Suma de muestra total últimos meses', 'Autos por versión', 'promedio Q2_EBC', 'promedio Q2_ATM', 'Muestra suficiente', 'factor EBC ajustado', 'factor ATM ajustado', 'fuera del límite EBC', 'Factor de EBC negativo', 'debajo del límite EBC', 'arriba del límite EBC', 'fuera del límite ATM', 'Factor de ATM negativo', 'debajo del límite ATM', 'arriba del límite ATM']

# Convertir listas a conjuntos
set_mm = set(lista_mm)
set_mma = set(lista_mma)

# Encontrar valores diferentes entre los conjuntos
diferencia_mm_mma = set_mm - set_mma
diferencia_mma_mm = set_mma - set_mm

# Mostrar los valores diferentes
print("Valores diferentes en lista_mm con respecto a lista_mma:", diferencia_mm_mma)
print("Valores diferentes en lista_mma con respecto a lista_mm:", diferencia_mma_mm)


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


Valores diferentes en lista_mm con respecto a lista_mma: {'muestra total últimos meses', 'EbcId', 'emulado por versiones hermanas', 'trim', 'emulado con EBC', 'Fecha', 'AutometricaId', 'categoryId', 'priceSaleGam', 'MP_vs_ATM', 'pricePurchaseGam', 'muestras por mes', 'MP_vs_EBC', 'MaxiPublica', 'priceSaleEbc', 'emulado con versión anterior', 'muestra total', 'pricePurchaseEbc'}
Valores diferentes en lista_mma con respecto a lista_mm: {'% EBC superior', 'arriba del límite ATM', 'Q0_EBC', 'llave_mm', 'factor ATM ajustado', 'fuera del límite EBC', '% ATM superior', 'Muestra suficiente', 'Q1_EBC', 'límite superior EBC', 'Q3_ATM', 'ATM con outlier superior', 'factor EBC ajustado', 'promedio Q2_ATM', 'Q2_ATM', '% EBC inferior', 'Q0_ATM', 'EBC con outlier superior', 'Q4_EBC', 'debajo del límite EBC', 'límite superior ATM', 'debajo del límite ATM', 'límite inferior EBC', 'Q1_ATM', 'EBC con outlier inferior', 'total elementos', 'ATM con outlier inferior', 'Q2_EBC', '% ATM inferior', 'Factor de 

In [6]:
#df.columns
df_mma.columns

Index(['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC',
       'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'EBC con outlier inferior', 'EBC con outlier superior',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por versión',
       'promedio Q2_EBC', 'promedio Q2_ATM'],
      dtype='object')

#### Trasforma los datos leidos al tipo de dato esperado

In [7]:
# Trasforma los datos leidos al tipo correcto

# Eliminar la columna 'Unnamed: 0'
#df = df.drop('Unnamed: 0', axis=1)

# Convertir a booleano
columnas_a_convertir = ['emulado con versión anterior', 'emulado con EBC', 'emulado por versiones hermanas']

# Cambiar el tipo de datos a booleano para las columnas especificadas
df[columnas_a_convertir] = df[columnas_a_convertir].astype(bool)

# Convertir los valores de 'model' a strings
df['model'] = df['model'].astype(str)

# Convertir la columna 'Fecha' a tipo datetime
df['Fecha']  = pd.to_datetime(df['Fecha'])

# Formatear la columna 'Fecha' para mostrar solo la parte de la fecha
df['Fecha'] = df['Fecha'].dt.strftime('%Y-%m-%d')

# Ordena el DataFrame por las columnas indicadas
df  =  df.sort_values(by=[ 'categoryId',  'Fecha'])

# Crear una llave por marca, modelo, año
df['llave_mma'] = df['brand'] + '_' + df['model'] + '_' + df['year'].astype(str)

# Resetea los índices del DataFrame
df  =  df.reset_index(drop=True)


nombres_columnas_h = df.columns.tolist()
print("Nombres de las columnas: df datos_historicos")
print(df.columns)

display(df.head())
#display(df.columns)
#df.to_csv(f'{ruta_local_datos_temporal}/00_datos_historicos.csv', sep=',')


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


Nombres de las columnas: df datos_historicos
Index(['Fecha', 'categoryId', 'brand', 'model', 'year', 'trim', 'MaxiPublica',
       'muestras por mes', 'muestra total últimos meses', 'muestra total',
       'emulado con versión anterior', 'emulado con EBC',
       'emulado por versiones hermanas', 'priceSaleEbc', 'pricePurchaseEbc',
       'priceSaleGam', 'pricePurchaseGam', 'EbcId', 'AutometricaId',
       'llave_mma', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por versión', 'MP_vs_EBC',
       'MP_vs_ATM'],
      dtype='object')


,Fecha,categoryId,brand,model,year,trim,MaxiPublica,muestras por mes,muestra total últimos meses,muestra total,...,priceSaleGam,pricePurchaseGam,EbcId,AutometricaId,llave_mma,Cantidad de versiones por año,Suma de muestra total últimos meses,Autos por versión,MP_vs_EBC,MP_vs_ATM
0,2023-09-01,v_1_10_13_1_1,Cadillac,Escalade,2015,5p ESV Premium D V8/6.2 Aut,662600.0,0,0,4,...,632400.0,545600.0,v_1_10_4_8_3,v_1_7_5_2_5,Cadillac_Escalade_2015,8,23,2.875,0.144280,0.045578
1,2023-10-01,v_1_10_13_1_1,Cadillac,Escalade,2015,5p ESV Premium D V8/6.2 Aut,662600.0,0,0,4,...,636400.0,549100.0,v_1_10_4_8_3,v_1_7_5_2_5,Cadillac_Escalade_2015,8,23,2.875,0.144280,0.039541
2,2023-11-01,v_1_10_13_1_1,Cadillac,Escalade,2015,5p ESV Premium D V8/6.2 Aut,662600.0,0,0,4,...,640300.0,550300.0,v_1_10_4_8_3,v_1_7_5_2_5,Cadillac_Escalade_2015,8,23,2.875,0.145035,0.033655
3,2023-09-01,v_1_10_13_1_2,Cadillac,Escalade,2015,5p ESV Premium E V8/6.2 Aut 8/Pas,678100.0,0,0,2,...,690200.0,595500.0,v_1_10_4_8_4,v_1_7_5_2_6,Cadillac_Escalade_2015,8,23,2.875,0.160891,-0.017844
4,2023-10-01,v_1_10_13_1_2,Cadillac,Escalade,2015,5p ESV Premium E V8/6.2 Aut 8/Pas,678100.0,0,0,2,...,694600.0,599300.0,v_1_10_4_8_4,v_1_7_5_2_6,Cadillac_Escalade_2015,8,23,2.875,0.160891,-0.024333


Terminé: 2023-12-12 20:52:57 UTC-06:00


In [8]:
#df
df_mma.columns

Index(['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC',
       'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'EBC con outlier inferior', 'EBC con outlier superior',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por versión',
       'promedio Q2_EBC', 'promedio Q2_ATM'],
      dtype='object')

#Propuesta de solución

## Análisis de cuantiles de mma

In [9]:
# Establecer la variable 'muestra_minima_suficiente'
muestra_minima_suficiente = 10

# Agregar una columna 'Muestra suficiente' con valores booleanos
df_mma['Muestra suficiente'] = df_mma['Autos por versión'] > muestra_minima_suficiente

# Condición para llenar 'factor EBC ajustado'
condicion = df_mma['Autos por versión'] > muestra_minima_suficiente

# Asignar los valores según la condición
df_mma['factor EBC ajustado'] = np.where(condicion, df_mma['Q2_EBC'], df_mma['promedio Q2_EBC'])
df_mma['factor ATM ajustado'] = np.where(condicion, df_mma['Q2_ATM'], df_mma['promedio Q2_ATM'])






In [10]:
print(df_mma.columns)
display(df_mma[['llave_mma','Autos por versión','Q2_EBC']].head(2))

Index(['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC',
       'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'EBC con outlier inferior', 'EBC con outlier superior',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por versión',
       'promedio Q2_EBC', 'promedio Q2_ATM', 'Muestra suficiente',
       'factor EBC ajustado', 'factor ATM ajustado'],
      dtype='object')


,llave_mma,Autos por versión,Q2_EBC
0,Abarth_500_2014,4.0,0.223595
1,Abarth_500_2015,1.0,0.284571


## Calculando los percentiles y límites para factor EBC ajustado y factor ATM ajustado

In [11]:

# Calculando los percentiles y límites para factor EBC ajustado y factor ATM ajustado
Q0_faEBC = df_mma['factor EBC ajustado'].quantile(0)
Q1_faEBC = df_mma['factor EBC ajustado'].quantile(0.25)
Q2_faEBC = df_mma['factor EBC ajustado'].quantile(0.5)
Q3_faEBC = df_mma['factor EBC ajustado'].quantile(0.75)
Q4_faEBC = df_mma['factor EBC ajustado'].quantile(1)
IQR_EBC = Q3_faEBC - Q1_faEBC
limite_inferior_EBC = Q1_faEBC - 1.5 * IQR_EBC
limite_superior_EBC = Q3_faEBC + 1.5 * IQR_EBC

Q0_faATM = df_mma['factor ATM ajustado'].quantile(0)
Q1_faATM = df_mma['factor ATM ajustado'].quantile(0.25)
Q2_faATM = df_mma['factor ATM ajustado'].quantile(0.5)
Q3_faATM = df_mma['factor ATM ajustado'].quantile(0.75)
Q4_faATM = df_mma['factor ATM ajustado'].quantile(1)
IQR_ATM = Q3_faATM - Q1_faATM
limite_inferior_ATM = Q1_faATM - 1.5 * IQR_ATM
limite_superior_ATM = Q3_faATM + 1.5 * IQR_ATM

# Creando un DataFrame con los resultados
data = {
    'factor EBC ajustado': [Q0_faEBC, limite_inferior_EBC, Q1_faEBC, Q2_faEBC, Q3_faEBC, limite_superior_EBC, Q4_faEBC],
    'factor ATM ajustado': [Q0_faATM, limite_inferior_ATM, Q1_faATM, Q2_faATM, Q3_faATM, limite_superior_ATM, Q4_faATM]
}

index = ['Q0', 'lower_bound', 'Q1', 'Q2', 'Q3', 'upper_bound', 'Q4']

resumen_cuantiles = pd.DataFrame(data, index=index)
display(resumen_cuantiles)
resumen_cuantiles.to_csv(f'{ruta_local_datos_temporal}/07_resumen_cuantiles_ajustados_vs_libros.csv', sep=',')


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


,factor EBC ajustado,factor ATM ajustado
Q0,-0.339892,-0.633208
lower_bound,0.098079,-0.079756
Q1,0.168066,0.057610
Q2,0.189665,0.106959
Q3,0.214724,0.149187
upper_bound,0.284711,0.286552
Q4,0.393338,0.465147


Terminé: 2023-12-12 20:52:57 UTC-06:00


In [12]:
print(f'limite_inferior_EBC= {limite_inferior_EBC}')
print(f'limite_superior_EBC= {limite_superior_EBC}')


limite_inferior_EBC= 0.0980787522717414
limite_superior_EBC= 0.2847113191575598


In [13]:
# Agregar una columna 'fuera del límite EBC' con valores booleanos
df_mma['fuera del límite EBC'] = ((df_mma['factor EBC ajustado'] < limite_inferior_EBC) | (df_mma['factor EBC ajustado'] > limite_superior_EBC))
df_mma['Factor de EBC negativo'] = df_mma['factor EBC ajustado'] < 0
df_mma['debajo del límite EBC']  = df_mma['factor EBC ajustado'] < limite_inferior_EBC
df_mma['arriba del límite EBC']  = df_mma['factor EBC ajustado'] > limite_superior_EBC

# Agregar una columna 'fuera del límite ATM' con valores booleanos
df_mma['fuera del límite ATM'] = ((df_mma['factor ATM ajustado'] < limite_inferior_ATM) | (df_mma['factor ATM ajustado'] > limite_superior_ATM))
df_mma['Factor de ATM negativo'] = df_mma['factor ATM ajustado']<0
df_mma['debajo del límite ATM']  = df_mma['factor ATM ajustado'] < limite_inferior_ATM
df_mma['arriba del límite ATM']  = df_mma['factor ATM ajustado'] > limite_superior_ATM

df_mma.to_csv(f'{ruta_local_datos_temporal}/08_atípico_mm_ajustados_vs_libros.csv', sep=',', index=False)
df_mma.to_excel(f'{ruta_local_datos_temporal}/08_atípico_mm_ajustados_vs_libros.xlsx', index=False)
df_mma.head(5)


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


Terminé: 2023-12-12 20:53:06 UTC-06:00


In [14]:
df_mma.columns

Index(['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC',
       'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'EBC con outlier inferior', 'EBC con outlier superior',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por versión',
       'promedio Q2_EBC', 'promedio Q2_ATM', 'Muestra suficiente',
       'factor EBC ajustado', 'factor ATM ajustado', 'fuera del límite EBC',
       'Factor de EBC negativo', 'debajo del límite EBC',
       'arriba del límite EBC', 'fuera del límite ATM',
       'Factor de ATM negativo', 'debajo del límite ATM',
       'arriba del límite ATM'],
      dtype='object')

##Cálculo de factor promedio a nivel marca

### Obtiene el factor promedio vs EBC dentro de rango y con muestra suficiente.

In [15]:
# A partir de los elementos de los que se tiene suficiente muestra y los valores estan dentro de rango
# Obtiene las filas de los registros dentro de rango y con muestra suficiente.
#df_mma.columns

# Definir una función para calcular el promedio ponderado
def weighted_average(group):
    return (group['factor EBC ajustado'] * group['Autos por versión']).sum() / group['Autos por versión'].sum()

# Agrupar por 'brand' y aplicar la función de promedio ponderado
df_brand_factor_sin_filtro = df_mma.groupby('brand').apply(weighted_average)

# Convertir la serie de resultados en un DataFrame
df_brand_factor_sin_filtro = df_brand_factor_sin_filtro.reset_index()

# Nombrar las columnas del nuevo DataFrame
df_brand_factor_sin_filtro.columns = ['brand', 'EBC factor marca sin filtro']

# Filtrar las filas que que estan dentro del límite y tiene mas de muestra_minima_suficiente anuncios de muestra
condicion = (df_mma['fuera del límite EBC'] == False) & (df_mma['Autos por versión'] > muestra_minima_suficiente)
df_m = df_mma.loc[condicion]

# Mostrar las primeras filas del nuevo DataFrame df_m
display(df_m[['brand', 'model', 'year','Q2_EBC', 'Autos por versión', 'factor EBC ajustado','fuera del límite EBC']].head(3))

# Agrupar por 'brand' y aplicar la función de promedio ponderado
resultados = df_m.groupby('brand').apply(weighted_average)

# Convertir la serie de resultados en un DataFrame
df_brand_factor = resultados.reset_index()

# Nombrar las columnas del nuevo DataFrame
df_brand_factor.columns = ['brand', 'EBC factor marca con muestra suficiente']

# Realizar un merge utilizando la columna 'brand' como clave de combinación
df_brand_factor_ponderado_EBC = pd.merge(df_brand_factor_sin_filtro, df_brand_factor, on='brand', how='outer')

# Mostrar el DataFrame combinado
display(df_brand_factor_ponderado_EBC)


df_brand_factor_ponderado_EBC.to_csv(f'{ruta_local_datos_temporal}/08_brand_factor_ponderado_EBC.csv', sep=',', index=False)

df_brand_factor_ponderado_EBC.columns


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


,brand,model,year,Q2_EBC,Autos por versión,factor EBC ajustado,fuera del límite EBC
7,Acura,ILX,2015,0.180013,15.5,0.180013,False
8,Acura,ILX,2016,0.209029,40.0,0.209029,False
9,Acura,ILX,2017,0.219279,26.5,0.219279,False


,brand,EBC factor marca sin filtro,EBC factor marca con muestra suficiente
0,Abarth,0.214437,NaN
1,Acura,0.177408,0.187344
2,Alfa romeo,0.218736,0.247641
3,Audi,0.179954,0.187970
4,Baic,0.220944,NaN
5,Bmw,0.192619,0.193999
6,Buick,0.179886,0.170219
7,Cadillac,0.153771,NaN
8,Chevrolet,0.183155,0.181506
9,Chirey,0.177921,0.177921


Terminé: 2023-12-12 20:53:06 UTC-06:00


### Obtiene el factor promedio vs ATM dentro de rango y con muestra suficiente.

In [16]:
# A partir de los elementos de los que se tiene suficiente muestra y los valores estan dentrod e rango
# Obtiene las filas de los registros dentro de rango y con muestra suficiente.
#df_mma.columns

# Definir una función para calcular el promedio ponderado
def weighted_average(group):
    return (group['factor ATM ajustado'] * group['Autos por versión']).sum() / group['Autos por versión'].sum()

# Agrupar por 'brand' y aplicar la función de promedio ponderado
df_brand_factor_sin_filtro = df_mma.groupby('brand').apply(weighted_average)

# Convertir la serie de resultados en un DataFrame
df_brand_factor_sin_filtro = df_brand_factor_sin_filtro.reset_index()

# Nombrar las columnas del nuevo DataFrame
df_brand_factor_sin_filtro.columns = ['brand', 'ATM factor marca sin filtro']

# Filtrar las filas que que estan dentro del límite y tiene mas de muestra_minima_suficiente anuncios de muestra
condicion = (df_mma['fuera del límite ATM'] == False) & (df_mma['Autos por versión'] > muestra_minima_suficiente)
df_m = df_mma.loc[condicion]

# Mostrar las primeras filas del nuevo DataFrame df_m
display(df_m[['brand', 'model', 'year','Q2_ATM', 'Autos por versión', 'factor ATM ajustado','fuera del límite ATM']].head(3))

# Agrupar por 'brand' y aplicar la función de promedio ponderado
resultados = df_m.groupby('brand').apply(weighted_average)

# Convertir la serie de resultados en un DataFrame
df_brand_factor = resultados.reset_index()

# Nombrar las columnas del nuevo DataFrame
df_brand_factor.columns = ['brand', 'ATM factor marca con muestra suficiente']

# Realizar un merge utilizando la columna 'brand' como clave de combinación
df_brand_factor_ponderado_ATM = pd.merge(df_brand_factor_sin_filtro, df_brand_factor, on='brand', how='outer')

# Mostrar el DataFrame combinado
display(df_brand_factor_ponderado_ATM)


df_brand_factor_ponderado_ATM.to_csv(f'{ruta_local_datos_temporal}/08_brand_factor_ponderado_ATM.csv', sep=',', index=False)

df_brand_factor_ponderado_ATM.columns


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


,brand,model,year,Q2_ATM,Autos por versión,factor ATM ajustado,fuera del límite ATM
7,Acura,ILX,2015,0.141074,15.5,0.141074,False
8,Acura,ILX,2016,0.106819,40.0,0.106819,False
9,Acura,ILX,2017,0.081917,26.5,0.081917,False


,brand,ATM factor marca sin filtro,ATM factor marca con muestra suficiente
0,Abarth,0.161661,NaN
1,Acura,0.073043,0.090922
2,Alfa romeo,0.121464,0.104776
3,Audi,0.095274,0.111983
4,Baic,0.059187,NaN
5,Bmw,0.092972,0.096593
6,Buick,0.040383,0.041927
7,Cadillac,-0.000919,NaN
8,Chevrolet,0.068761,0.077890
9,Chirey,0.108235,0.108235


Terminé: 2023-12-12 20:53:07 UTC-06:00


### Factor del promedio ponderado global de EBC

In [17]:
# Factor del promedio ponderado global de EBC
# Calcular el producto entre 'factor EBC ajustado' y 'Autos por versión'
producto = df_mma['factor EBC ajustado'] * df_mma['Autos por versión']

# Calcular la suma del producto
suma_producto = producto.sum()

# Calcular la suma de 'Autos por versión'
suma_autos_por_version = df_mma['Autos por versión'].sum()

# Calcular el promedio ponderado
EBC_promedio_global_sin_filtro = suma_producto / suma_autos_por_version

print("El promedio EBC todo 'factor EBC ajustado':", EBC_promedio_global_sin_filtro)

# Factor del promedio ponderado global de EBC
# Calcular el producto entre 'factor EBC ajustado' y 'Autos por versión'
producto = df_m['factor EBC ajustado'] * df_m['Autos por versión']

# Calcular la suma del producto
suma_producto = producto.sum()

# Calcular la suma de 'Autos por versión'
suma_autos_por_version = df_m['Autos por versión'].sum()

# Calcular el promedio ponderado
EBC_promedio_global_con_filtro = suma_producto / suma_autos_por_version

print("El promedio filtrado 'factor EBC ajustado':", EBC_promedio_global_con_filtro)


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


El promedio EBC todo 'factor EBC ajustado': 0.1863155627319875
El promedio filtrado 'factor EBC ajustado': 0.18601529465508237
Terminé: 2023-12-12 20:53:07 UTC-06:00


### Factor del promedio ponderado global de ATM

In [18]:
# Factor del promedio ponderado global de ATM
# Calcular el producto entre 'factor ATM ajustado' y 'Autos por versión'
producto = df_mma['factor ATM ajustado'] * df_mma['Autos por versión']

# Calcular la suma del producto
suma_producto = producto.sum()

# Calcular la suma de 'Autos por versión'
suma_autos_por_version = df_mma['Autos por versión'].sum()

# Calcular el promedio ponderado
ATM_promedio_global_sin_filtro = suma_producto / suma_autos_por_version

print("El promedio ATM todo 'factor ATM ajustado':", ATM_promedio_global_sin_filtro)

# Factor del promedio ponderado global de ATM
# Calcular el producto entre 'factor ATM ajustado' y 'Autos por versión'
producto = df_m['factor ATM ajustado'] * df_m['Autos por versión']

# Calcular la suma del producto
suma_producto = producto.sum()

# Calcular la suma de 'Autos por versión'
suma_autos_por_version = df_m['Autos por versión'].sum()

# Calcular el promedio ponderado
ATM_promedio_global_con_filtro = suma_producto / suma_autos_por_version

print("El promedio filtrado 'factor ATM ajustado':", ATM_promedio_global_con_filtro)

El promedio ATM todo 'factor ATM ajustado': 0.0963855755504743
El promedio filtrado 'factor ATM ajustado': 0.10541086103958286


### Guarda los factores globales de los libros

In [19]:
import pandas as pd

# Definir las variables con los valores


# Crear un diccionario con los nombres de las columnas y los valores de las variables
data = {
    'Factores': ['EBC_promedio_global_sin_filtro', 'EBC_promedio_global_con_filtro',
                  'ATM_promedio_global_sin_filtro', 'ATM_promedio_global_con_filtro'],
    'Valores': [EBC_promedio_global_sin_filtro, EBC_promedio_global_con_filtro,
                ATM_promedio_global_sin_filtro, ATM_promedio_global_con_filtro]
}

# Crear el DataFrame
df_factor_global_libros = pd.DataFrame(data)

# Guardar el DataFrame en un archivo si es necesario
df_factor_global_libros.to_csv(f'{ruta_local_datos_temporal}/08_factor_global_libros.csv', sep=',', index=False)

# Mostrar el DataFrame creado
print(df_factor_global_libros)


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))



                         Factores   Valores
0  EBC_promedio_global_sin_filtro  0.186316
1  EBC_promedio_global_con_filtro  0.186015
2  ATM_promedio_global_sin_filtro  0.096386
3  ATM_promedio_global_con_filtro  0.105411
Terminé: 2023-12-12 20:53:07 UTC-06:00


### Agrega los factores por marca y globales al df_mma

---



### Agrega factores de niveles: año, modelo, marca

#### Renombra columnas

In [20]:
#Agrega factores de niveles: año, modelo, marca

# Crear un diccionario con los nuevos nombres de columnas
nuevos_nombres = {
    'promedio Q2_EBC': 'factor EBC marca modelo',
    'factor EBC ajustado': 'factor EBC marca modelo año',
    'promedio Q2_ATM': 'factor ATM marca modelo',
    'factor ATM ajustado': 'factor ATM marca modelo año',
    'Autos por versión' : 'Autos por marca modelo año'
}

# Renombrar las columnas usando el método rename
df_mma = df_mma.rename(columns=nuevos_nombres)

# Mostrar el DataFrame con las columnas renombradas
display(df_mma.head())


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


,brand,model,year,Q0_EBC,límite inferior EBC,Q1_EBC,Q2_EBC,Q3_EBC,límite superior EBC,Q4_EBC,...,factor EBC marca modelo año,factor ATM marca modelo año,fuera del límite EBC,Factor de EBC negativo,debajo del límite EBC,arriba del límite EBC,fuera del límite ATM,Factor de ATM negativo,debajo del límite ATM,arriba del límite ATM
0,Abarth,500,2014,0.221224,0.219881,0.222409,0.223595,0.224094,0.226622,0.224594,...,0.214437,0.161661,False,False,False,False,False,False,False,False
1,Abarth,500,2015,0.258990,0.214592,0.259043,0.284571,0.288677,0.333128,0.292828,...,0.214437,0.161661,False,False,False,False,False,False,False,False
2,Abarth,500,2016,0.185455,0.020121,0.191034,0.258319,0.304975,0.475888,0.351222,...,0.214437,0.161661,False,False,False,False,False,False,False,False
3,Abarth,500,2017,0.268447,0.195875,0.268550,0.311612,0.317001,0.389676,0.319820,...,0.214437,0.161661,False,False,False,False,False,False,False,False
4,Abarth,500,2019,0.141971,0.104933,0.145202,0.148220,0.172048,0.212316,0.182313,...,0.214437,0.161661,False,False,False,False,False,False,False,False


Terminé: 2023-12-12 20:53:07 UTC-06:00


### Agrega los factores de marca al df_mma

### Factores a nivel marca y modelo

In [21]:
#display(df_mma.columns)

#agrega los factores a nivel marca para el libro EBC
# Realizar un merge utilizando la columna 'brand' como clave de combinación y 'left' para conservar todas las filas de df_mma
df_mma = pd.merge(df_mma, df_brand_factor_ponderado_EBC, on='brand', how='left')

#agrega los factores a nivel marca para el libro ATM
# Realizar un merge utilizando la columna 'brand' como clave de combinación y 'left' para conservar todas las filas de df_mma
df_mma = pd.merge(df_mma, df_brand_factor_ponderado_ATM, on='brand', how='left')



# Mostrar el DataFrame combinado
display(df_mma.head())


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))



,brand,model,year,Q0_EBC,límite inferior EBC,Q1_EBC,Q2_EBC,Q3_EBC,límite superior EBC,Q4_EBC,...,debajo del límite EBC,arriba del límite EBC,fuera del límite ATM,Factor de ATM negativo,debajo del límite ATM,arriba del límite ATM,EBC factor marca sin filtro,EBC factor marca con muestra suficiente,ATM factor marca sin filtro,ATM factor marca con muestra suficiente
0,Abarth,500,2014,0.221224,0.219881,0.222409,0.223595,0.224094,0.226622,0.224594,...,False,False,False,False,False,False,0.214437,NaN,0.161661,NaN
1,Abarth,500,2015,0.258990,0.214592,0.259043,0.284571,0.288677,0.333128,0.292828,...,False,False,False,False,False,False,0.214437,NaN,0.161661,NaN
2,Abarth,500,2016,0.185455,0.020121,0.191034,0.258319,0.304975,0.475888,0.351222,...,False,False,False,False,False,False,0.214437,NaN,0.161661,NaN
3,Abarth,500,2017,0.268447,0.195875,0.268550,0.311612,0.317001,0.389676,0.319820,...,False,False,False,False,False,False,0.214437,NaN,0.161661,NaN
4,Abarth,500,2019,0.141971,0.104933,0.145202,0.148220,0.172048,0.212316,0.182313,...,False,False,False,False,False,False,0.214437,NaN,0.161661,NaN


Terminé: 2023-12-12 20:53:07 UTC-06:00


#### Cálculo de anuncios promedio por MARCA-MODELO

In [22]:
# Cálculo de anuncios promedio por marca modelo

# Calcula el promedio de 'Autos por marca modelo año' por cada grupo 'llave_mm'
promedio_autos_por_version = df_mma.groupby('llave_mm')['Autos por marca modelo año'].mean()

# Agrega esta información al DataFrame original basándote en 'llave_mm'
df_mma['Autos por marca modelo'] = df_mma['llave_mm'].map(promedio_autos_por_version)

# Muestra el DataFrame con la nueva columna 'factor Autos por marca modelo'
display(df_mma[['llave_mma', 'Cantidad de versiones por año',
                'Suma de muestra total últimos meses', 'Autos por marca modelo año',
                'Autos por marca modelo']]
        .sort_values('llave_mma')
        .head(11))


,llave_mma,Cantidad de versiones por año,Suma de muestra total últimos meses,Autos por marca modelo año,Autos por marca modelo
0,Abarth_500_2014,1,4,4.000000,2.366667
1,Abarth_500_2015,3,3,1.000000,2.366667
2,Abarth_500_2016,3,2,0.666667,2.366667
3,Abarth_500_2017,3,5,1.666667,2.366667
4,Abarth_500_2019,2,9,4.500000,2.366667
5,Acura_ILX_2013,2,2,1.000000,22.812500
6,Acura_ILX_2014,3,12,4.000000,22.812500
7,Acura_ILX_2015,2,31,15.500000,22.812500
8,Acura_ILX_2016,2,80,40.000000,22.812500
9,Acura_ILX_2017,2,53,26.500000,22.812500


#### Cálculo de anuncios promedio por MARCA-MODELO

In [23]:
df_mma.columns

Index(['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC',
       'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'EBC con outlier inferior', 'EBC con outlier superior',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por marca modelo año',
       'factor EBC marca modelo', 'factor ATM marca modelo',
       'Muestra suficiente', 'factor EBC marca modelo año',
       'factor ATM marca modelo año', 'fuera del límite EBC',
       'Factor de EBC negativo', 'debajo del límite EBC',
       'arriba del límite EBC', 'fuera del límite ATM',
       'Factor de ATM negativo', 'debajo del límite ATM',
       'arriba del límite ATM', 'E

In [24]:
# Cálculo de anuncios promedio por marca

# Calcula el promedio de 'Autos por marca modelo año' por cada grupo 'llave_mm'
promedio_autos_por_marca = df_mma.groupby('brand')['Autos por marca modelo año'].mean()

# Agrega esta información al DataFrame original basándote en 'llave_mm'
df_mma['Autos por marca'] = df_mma['brand'].map(promedio_autos_por_marca)

# Muestra el DataFrame con la nueva columna 'factor Autos por marca modelo'


display(df_mma[['llave_mma', 'Muestra suficiente',
                'Autos por marca modelo año',
                'Autos por marca modelo','Autos por marca',
                'factor EBC marca modelo año', 'factor EBC marca modelo',
                'EBC factor marca con muestra suficiente'
                ]]
        .sort_values('llave_mma')
        .iloc[1:20])


,llave_mma,Muestra suficiente,Autos por marca modelo año,Autos por marca modelo,Autos por marca,factor EBC marca modelo año,factor EBC marca modelo,EBC factor marca con muestra suficiente
1,Abarth_500_2015,False,1.000000,2.366667,2.366667,0.214437,0.214437,NaN
2,Abarth_500_2016,False,0.666667,2.366667,2.366667,0.214437,0.214437,NaN
3,Abarth_500_2017,False,1.666667,2.366667,2.366667,0.214437,0.214437,NaN
4,Abarth_500_2019,False,4.500000,2.366667,2.366667,0.214437,0.214437,NaN
5,Acura_ILX_2013,False,1.000000,22.812500,11.488095,0.199059,0.199059,0.187344
6,Acura_ILX_2014,False,4.000000,22.812500,11.488095,0.199059,0.199059,0.187344
7,Acura_ILX_2015,True,15.500000,22.812500,11.488095,0.180013,0.199059,0.187344
8,Acura_ILX_2016,True,40.000000,22.812500,11.488095,0.209029,0.199059,0.187344
9,Acura_ILX_2017,True,26.500000,22.812500,11.488095,0.219279,0.199059,0.187344
10,Acura_ILX_2018,True,46.500000,22.812500,11.488095,0.173707,0.199059,0.187344


In [25]:
#df_mma.columns

### Se afrega el 'factor final EBC'

#### ini Aux. calculos

##### Filtrar MMA con 'muestra suficiente' y 'fuera del límite inferior de EBC'

##### Reporte de auncios con muestra suficiente y fuerá de límite inferior

In [26]:
# Reporte de auncios con muestra suficiente y fuerá de límite inferior

# Filtrar el muestra suficiente de MMA fuera del límite inferior de EBC
condicion = ((df_mma['Autos por marca modelo año'] > muestra_minima_suficiente)
            & (df_mma['factor EBC marca modelo año'] < limite_inferior_EBC))
resultados = df_mma[condicion]

# Agregar la columna 'Acción' con contenido constante
resultados = resultados.assign(Acción='Revisar la homologación de las versiones con los libros y/o la clasificación de anuncios en MercadoLobre')

display(resultados[['llave_mma', 'Muestra suficiente',
                'Autos por marca modelo año',
                'Autos por marca modelo','Autos por marca',
                'factor EBC marca modelo año', 'factor EBC marca modelo',
                'EBC factor marca con muestra suficiente'
                ]])

# Guardar el DataFrame en un archivo si es necesario
resultados.to_csv(f'{ruta_local_datos_temporal}/r01_Revisar_homologación_identificación.csv', sep=',', index=False)


,llave_mma,Muestra suficiente,Autos por marca modelo año,Autos por marca modelo,Autos por marca,factor EBC marca modelo año,factor EBC marca modelo,EBC factor marca con muestra suficiente
33,Acura_RDX_2021,True,16.500000,12.681818,11.488095,0.073871,0.165374,0.187344
195,Audi_S4_2016,True,15.000000,7.333333,6.274625,-0.339892,-0.142986,0.187970
907,Ford_F 250_2023,True,24.000000,8.727273,9.398481,0.031879,0.135882,0.159624
1011,Ford_Transit Van_2013,True,14.000000,2.965000,9.398481,-0.101843,0.022320,0.159624
1364,Jeep_Renegade_2023,True,22.500000,21.169048,13.910740,0.096399,0.194341,0.185802
1984,Nissan_Pathfinder_2017,True,28.000000,16.461111,15.525519,0.085746,0.125092,0.177988
2056,Nissan_X Trail_2023,True,10.166667,37.612229,15.525519,0.035627,0.150747,0.177988


###### Define  'factor final EBC' =  'factor EBC marca modelo' para en autos con muestra sufucuente a nivel modelo y factor dentro de límites

In [27]:
# Define  'factor final EBC' =  'factor EBC marca modelo' para en autos con
# muestra sufucuente a nivel modelo y factor dentro de límites
condicion_1 = ((df_mma['Autos por marca modelo año'] > muestra_minima_suficiente)
  & (df_mma['factor EBC marca modelo año'] < limite_inferior_EBC)
  & (df_mma['Autos por marca modelo'] > muestra_minima_suficiente)
  & (df_mma['factor EBC marca modelo'] > limite_inferior_EBC)
  )
resultados = df_mma[condicion_1]

display(resultados[['llave_mma', 'Muestra suficiente',
                'Autos por marca modelo año',
                'Autos por marca modelo','Autos por marca',
                'factor EBC marca modelo año', 'factor EBC marca modelo',
                'EBC factor marca con muestra suficiente'
                ]])

,llave_mma,Muestra suficiente,Autos por marca modelo año,Autos por marca modelo,Autos por marca,factor EBC marca modelo año,factor EBC marca modelo,EBC factor marca con muestra suficiente
33,Acura_RDX_2021,True,16.500000,12.681818,11.488095,0.073871,0.165374,0.187344
1364,Jeep_Renegade_2023,True,22.500000,21.169048,13.910740,0.096399,0.194341,0.185802
1984,Nissan_Pathfinder_2017,True,28.000000,16.461111,15.525519,0.085746,0.125092,0.177988
2056,Nissan_X Trail_2023,True,10.166667,37.612229,15.525519,0.035627,0.150747,0.177988


In [28]:
condicion_2 = (
    (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente)
  & (df_mma['factor EBC marca modelo año'] < limite_inferior_EBC)
  & (df_mma['Autos por marca modelo'] < muestra_minima_suficiente)
  & (df_mma['EBC factor marca con muestra suficiente'] > limite_inferior_EBC)
  )
resultados = df_mma[condicion_2]

display(resultados[['llave_mma', 'Muestra suficiente',
                'Autos por marca modelo año',
                'Autos por marca modelo','Autos por marca',
                'factor EBC marca modelo año', 'factor EBC marca modelo',
                'EBC factor marca con muestra suficiente'
                ]])

,llave_mma,Muestra suficiente,Autos por marca modelo año,Autos por marca modelo,Autos por marca,factor EBC marca modelo año,factor EBC marca modelo,EBC factor marca con muestra suficiente
195,Audi_S4_2016,True,15.0,7.333333,6.274625,-0.339892,-0.142986,0.187970
907,Ford_F 250_2023,True,24.0,8.727273,9.398481,0.031879,0.135882,0.159624
1011,Ford_Transit Van_2013,True,14.0,2.965000,9.398481,-0.101843,0.022320,0.159624


#### fin Aux. Calculos

In [29]:
import numpy as np

# Paso 1: Crear una nueva columna 'factor final EBC' con valores vacíos
df_mma['factor final EBC'] = np.nan

# Paso 2: Aplicar las condiciones para asignar los valores en la columna 'factor final EBC'
condicion_1 = (
    (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente) &
    (df_mma['factor EBC marca modelo año'] < limite_inferior_EBC) &
    (df_mma['Autos por marca modelo'] > muestra_minima_suficiente) &
    (df_mma['factor EBC marca modelo'] > limite_inferior_EBC)
)

condicion_2 = (
    (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente) &
    (df_mma['factor EBC marca modelo año'] < limite_inferior_EBC) &
    (df_mma['Autos por marca modelo'] < muestra_minima_suficiente) &
    (df_mma['EBC factor marca con muestra suficiente'] > limite_inferior_EBC)
)

# Asignar valores en la columna 'factor final EBC' basados en las condiciones
df_mma.loc[condicion_1, 'factor final EBC'] = df_mma.loc[condicion_1, 'factor EBC marca modelo']
df_mma.loc[condicion_2, 'factor final EBC'] = df_mma.loc[condicion_2, 'EBC factor marca con muestra suficiente']

# Llenar los valores restantes con el valor de 'EBC_promedio_global_con_filtro'
##df_mma['factor final EBC'].fillna(EBC_promedio_global_con_filtro, inplace=True)


In [ ]:
### Pobla el nuevo 'factor final EBC'

In [31]:
import numpy as np

# Establecer las condiciones para poblar 'factor final EBC'
condicion_1 = (df_mma['factor EBC marca modelo año'].notnull()) & (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente)
condicion_2 = (df_mma['factor EBC marca modelo'].notnull()) & (df_mma['Autos por marca modelo'] > muestra_minima_suficiente)
condicion_3 = (df_mma['EBC factor marca con muestra suficiente'].notnull()) & (df_mma['Autos por marca'] > muestra_minima_suficiente)

# Aplicar las condiciones para poblar 'factor final EBC'
df_mma.loc[condicion_1, 'factor final EBC'] = df_mma.loc[condicion_1, 'factor EBC marca modelo año']
df_mma.loc[condicion_2 & df_mma['factor final EBC'].isnull(), 'factor final EBC'] = df_mma.loc[condicion_2 & df_mma['factor final EBC'].isnull(), 'factor EBC marca modelo']
df_mma.loc[condicion_3 & df_mma['factor final EBC'].isnull(), 'factor final EBC'] = df_mma.loc[condicion_3 & df_mma['factor final EBC'].isnull(), 'EBC factor marca con muestra suficiente']

# Rellenar los valores restantes con 'EBC_promedio_global_con_filtro'
df_mma['factor final EBC'].fillna(EBC_promedio_global_con_filtro, inplace=True)


### Se afrega el 'factor final ATM'

In [ ]:

# Suponiendo que ATM_promedio_global_con_filtro y df_mma ya están definidos

def calcular_factor_final(row):
    if row['Muestra suficiente'] == True and not pd.isnull(row['factor ATM marca modelo']):
        return row['factor ATM marca modelo']
    elif not pd.isnull(row['ATM factor marca con muestra suficiente']):
        return row['ATM factor marca con muestra suficiente']
    else:
        return ATM_promedio_global_con_filtro

# Aplicar la función a lo largo de las filas del DataFrame para crear la nueva columna
df_mma['factor final ATM'] = df_mma.apply(calcular_factor_final, axis=1)



# Mostrar el DataFrame con la nueva columna 'factor final ATM'
display(df_mma.head(5))


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))



### Guarda los resulados del factor final

In [ ]:
df_mma.to_csv(f'{ruta_local_datos_temporal}/08_mma_factores_niveles.csv', sep=',', index=False)
df_mma.to_excel(f'{ruta_local_datos_temporal}/08_mma_factores_niveles.xlsx', index=False)


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


## Filtros para análisis

In [ ]:
#df_mma.columns
#df.columns

In [ ]:
# Filtrar filas donde 'fuera del límite EBC' es True

print(f'limite_inferior_EBC= {limite_inferior_EBC}')
print(f'limite_superior_EBC= {limite_superior_EBC}')
#display(df_mma[((df_mma['fuera del límite EBC']) )][['brand', 'model', 'year','Q2_EBC', 'Autos por marca modelo año', 'factor EBC marca modelo año','fuera del límite EBC']])
#display(df_mma[((df_mma['Autos por marca modelo año']< muestra_minima_suficiente) )][['brand', 'model', 'year','Q2_EBC', 'Autos por marca modelo año', 'factor EBC marca modelo año','fuera del límite EBC']])
display(df_mma[((df_mma['fuera del límite EBC'])  & (df_mma['Autos por marca modelo año']< muestra_minima_suficiente) )][['brand', 'model', 'year','Q2_EBC', 'Autos por marca modelo año', 'factor EBC marca modelo año','fuera del límite EBC']])
#and  (df_mma['por versión']< muestra_minima_suficiente)


In [ ]:
# Filtrar filas donde 'fuera del límite ATM' es True

print(f'limite_inferior_ATM= {limite_inferior_ATM}')
print(f'limite_superior_ATM= {limite_superior_ATM}')
display(df_mma[df_mma['fuera del límite ATM']][['brand', 'model', 'year','Q2_ATM', 'factor ATM marca modelo año','fuera del límite ATM']])


### Filtrar y mostrar filas donde 'fuera del límite EBC' o 'fuera del límite ATM' son verdaderos

In [ ]:
# Filtrar y mostrar filas donde 'fuera del límite EBC' o 'fuera del límite ATM' son verdaderos

df_fuera_de_limite = df_mma[(df_mma['fuera del límite EBC']) | (df_mma['fuera del límite ATM'])]

filas_fuera_de_limite = len(df_fuera_de_limite)
# Mostrar la cantidad de filas del resultado
print("Cantidad de filas fuera de límite EBC y ATM:", filas_fuera_de_limite)


print(f'limite_inferior_EBC= {limite_inferior_EBC}')
print(f'limite_superior_EBC= {limite_superior_EBC}')
print(f'limite_inferior_ATM= {limite_inferior_ATM}')
print(f'limite_superior_ATM= {limite_superior_ATM}')


# Mostrar filas donde 'fuera del límite EBC' y 'fuera del límite ATM' son verdaderos
display(df_fuera_de_limite[['brand', 'model', 'year','Q2_EBC', 'factor EBC marca modelo año','Q2_ATM', 'factor ATM marca modelo año']])

df_mma.to_csv(f'{ruta_local_datos_temporal}/09_atípicos_mm_vs_libros.csv', sep=',', index=False)
df_mma.head(5)

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


In [ ]:
# Auxiliar para identificar las columnas diferentes y usarlas en el merge
cols2 = ['brand', 'model', 'year', 'Q0_EBC', 'límite inferior EBC', 'Q1_EBC',
       'Q2_EBC', 'Q3_EBC', 'límite superior EBC', 'Q4_EBC', 'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'EBC con outlier inferior', 'EBC con outlier superior',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% EBC inferior', '% EBC superior', '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por marca modelo año',
       'factor EBC marca modelo', 'factor ATM marca modelo', 'Muestra suficiente',
       'factor EBC marca modelo año', 'factor ATM marca modelo año', 'fuera del límite EBC',
       'Factor de EBC negativo', 'debajo del límite EBC',
       'arriba del límite EBC', 'fuera del límite ATM',
       'Factor de ATM negativo', 'debajo del límite ATM',
       'arriba del límite ATM']

cols1 = ['Fecha', 'categoryId', 'brand', 'model', 'year', 'trim', 'MaxiPublica',
       'muestras por mes', 'muestra total últimos meses', 'muestra total',
       'emulado con versión anterior', 'emulado con EBC',
       'emulado por versiones hermanas', 'priceSaleEbc', 'pricePurchaseEbc',
       'priceSaleGam', 'pricePurchaseGam', 'EbcId', 'AutometricaId',
       'llave_mma', 'Cantidad de versiones por año',
       'Suma de muestra total últimos meses', 'Autos por marca modelo año', 'MP_vs_EBC',
       'MP_vs_ATM']

cols1 = set(cols1)
cols2 = set(cols2)

cols_only_in_cols1 = cols2.difference(cols1)

print(cols_only_in_cols1)


#### Filtrar filas donde 'Factor de EBC negativo'

In [ ]:
# Filtrar filas donde 'Factor de EBC negativo' es True
df_EBC_negativo = df_mma[df_mma['Factor de EBC negativo'] == True].copy()

df_EBC_negativo.to_excel(f'{ruta_local_datos_temporal}/09_EBC_mma_negativo.xlsx', index=False)


# Lista de columnas a agregar de df_EBC_negativo a df
columnas_a_agregar = ['Q4_ATM', 'Q0_EBC', 'Q4_EBC', 'Factor de EBC negativo', 'Q0_ATM',
                      'fuera del límite ATM', 'factor EBC marca modelo', 'Muestra suficiente',
                      'Q3_ATM', 'Factor de ATM negativo', 'arriba del límite ATM',
                      'Q1_EBC', 'Q1_ATM', 'EBC con outlier inferior', '% ATM superior',
                      'ATM con outlier superior', 'factor ATM marca modelo año',
                      'Q3_EBC', '% EBC superior', '% EBC inferior', 'Q2_EBC', '% ATM inferior',
                      'factor ATM marca modelo', 'límite superior EBC',
                      'EBC con outlier superior', 'límite inferior ATM', 'total elementos',
                      'arriba del límite EBC', 'ATM con outlier inferior', 'llave_mm',
                      'debajo del límite EBC', 'fuera del límite EBC', 'Q2_ATM',
                      'límite superior ATM', 'límite inferior EBC',
                      'factor EBC marca modelo año', 'debajo del límite ATM']

# Realizar el merge y agregar las columnas seleccionadas
df_merged = pd.merge(df, df_EBC_negativo[columnas_a_agregar + ['llave_mma']], on='llave_mma', how='inner')

df_merged.to_csv(f'{ruta_local_datos_temporal}/09_EBC_negativo.csv', sep=',', index=False)
df_merged.to_excel(f'{ruta_local_datos_temporal}/09_EBC_negativo.xlsx', index=False)

# df_merged contendrá el DataFrame resultante con las columnas fusionadas
display(df_EBC_negativo.head())
#display(df_merged)
#print(df_EBC_negativo.size)
#display(df_EBC_negativo.head())
#print('df_EBC_negativo:', df_EBC_negativo.columns)
#print('df:', df.columns)

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


In [ ]:
display(df_EBC_negativo[['llave_mma', 'Q2_EBC','factor final EBC']].head())

In [ ]:
# Filtrar filas donde 'Factor de EBC negativo' es True
df_EBC_negativo = df_mma[((df_mma['Factor de EBC negativo'] == True) & (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente))].copy()
df_EBC_negativo['Observaciones'] = 'Revisar homologación'

df_EBC_negativo
df_EBC_negativo.to_excel(f'{ruta_local_datos_temporal}/09_EBC_mma_negativo_revisión.xlsx', index=False)


#### Filtrar filas donde 'Factor de ATM negativo'

In [ ]:
# Filtrar filas donde 'Factor de ATM negativo' es True
df_ATM_negativo = df_mma[df_mma['Factor de ATM negativo'] == True].copy()

df_ATM_negativo.to_excel(f'{ruta_local_datos_temporal}/09_ATM_mma_negativo.xlsx', index=False)


# Lista de columnas a agregar de df_ATM_negativo a df
columnas_a_agregar = ['Q4_ATM', 'Q0_ATM', 'Q4_ATM', 'Factor de ATM negativo', 'Q0_ATM', 'fuera del límite ATM', 'factor ATM marca modelo', 'Muestra suficiente', 'Q3_ATM', 'Factor de ATM negativo', 'arriba del límite ATM', 'Q1_ATM', 'Q1_ATM', 'ATM con outlier inferior', '% ATM superior', 'ATM con outlier superior', 'factor ATM marca modelo año', 'Q3_ATM', '% ATM superior', '% ATM inferior', 'Q2_ATM', '% ATM inferior', 'factor ATM marca modelo', 'límite superior ATM', 'ATM con outlier superior', 'límite inferior ATM', 'total elementos', 'arriba del límite ATM', 'ATM con outlier inferior', 'llave_mm', 'debajo del límite ATM', 'fuera del límite ATM', 'Q2_ATM', 'límite superior ATM', 'límite inferior ATM', 'factor ATM marca modelo año', 'debajo del límite ATM']

# Realizar el merge y agregar las columnas seleccionadas
df_merged = pd.merge(df, df_ATM_negativo[columnas_a_agregar + ['llave_mma']], on='llave_mma', how='inner')

df_merged.to_csv(f'{ruta_local_datos_temporal}/09_ATM_negativo.csv', sep=',', index=False)
df_merged.to_excel(f'{ruta_local_datos_temporal}/09_ATM_negativo.xlsx', index=False)

In [ ]:
# Filtrar filas donde 'Factor de ATM negativo' es True
df_ATM_negativo = df_mma[
    ((df_mma['Factor de ATM negativo'] == True) &
     (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente) &
     (df_mma['factor ATM marca modelo'].notna()))
].copy()
df_ATM_negativo['Observaciones'] = 'Revisar homologación'

df_ATM_negativo.to_excel(f'{ruta_local_datos_temporal}/09_ATM_mma_negativo_revisión.xlsx', index=False)
#df_ATM_negativo.columns
display(df_ATM_negativo[['brand', 'model', 'year',  'Q0_ATM',
       'límite inferior ATM', 'Q1_ATM', 'Q2_ATM', 'Q3_ATM',
       'límite superior ATM', 'Q4_ATM', 'total elementos',
       'ATM con outlier inferior', 'ATM con outlier superior',
       '% ATM inferior', '% ATM superior',
       'llave_mma', 'llave_mm', 'Cantidad de versiones por año',
       'Autos por marca modelo año',
        'factor ATM marca modelo',
       'factor EBC marca modelo año', 'factor ATM marca modelo año',
       'fuera del límite ATM',
       'Factor de ATM negativo', 'debajo del límite ATM',
       'arriba del límite ATM', 'Observaciones']])

### Filtrar filas donde 'Factor de EBC negativo' esta fuera de los límites inferior y superior

In [ ]:
# Filtrar filas donde 'Factor de EBC negativo' es True
df_EBC_negativo = df_mma[((df_mma['Factor de EBC negativo'] == True) & (df_mma['Autos por marca modelo año'] > muestra_minima_suficiente))].copy()
df_EBC_negativo['Observaciones'] = 'Revisar homologación'

df_EBC_negativo
df_EBC_negativo.to_excel(f'{ruta_local_datos_temporal}/09_EBC_mma_negativo_revisión.xlsx', index=False)


# Solución

## Agrega los factores de precio de libro al df original

In [ ]:
df['llave_mm'] = df['brand'] + '_' + df['model']
df.columns

In [ ]:
df_mma.columns

## Paso 1: Merge de 'df' y 'df_mma' por 'llave_mma'

In [ ]:
# Hacer un merge de 'df' y 'df_mma' por 'llave_mma'
df_merged = df.merge(df_mma[['llave_mma', 'factor final EBC', 'factor final ATM']], on='llave_mma', how='left')
display(df_merged.head())
filas_nan_price_published = df_merged[df_merged['factor final EBC'].isnull()]
display(filas_nan_price_published[['llave_mma','factor final EBC']])

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


## Paso 2: Llenar NaN en 'factor final EBC' con valores de 'df_mma' por 'llave_mm'

In [ ]:
# Identificar filas donde 'factor final EBC' es NaN y llenarlas con valores de 'df_mma' por 'llave_mm'
nan_indices = df_merged[df_merged['factor final EBC'].isnull()].index
for idx in nan_indices:
    llave_mm = df_merged.loc[idx, 'llave_mm']
    mma_row = df_mma[df_mma['llave_mm'] == llave_mm]
    if not mma_row.empty:
        df_merged.at[idx, 'factor final EBC'] = mma_row.iloc[0]['factor final EBC']
        df_merged.at[idx, 'factor final ATM'] = mma_row.iloc[0]['factor final ATM']


In [ ]:
filas_nan_price_published = df_merged[df_merged['factor final EBC'].isnull()]
display(filas_nan_price_published[['llave_mma','factor final EBC']])

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


## Paso 3: Llenar NaN restantes con 'EBC_promedio_global_con_filtro'

In [ ]:
# Identificar filas restantes donde 'factor final EBC' sigue siendo NaN y llenarlas con 'EBC_promedio_global_con_filtro'
remaining_nan_indices = df_merged[df_merged['factor final EBC'].isnull()].index
for idx in remaining_nan_indices:
    df_merged.at[idx, 'factor final EBC'] = EBC_promedio_global_con_filtro


In [ ]:
filas_nan_price_published = df_merged[df_merged['factor final EBC'].isnull()]
display(filas_nan_price_published[['llave_mma','factor final EBC']])

### Genera los precios de venta y compra para MaxiPublica

In [ ]:
# Genera los precios de venta y compra para MaxiPublica


# Calcular 'pricePublished' (precio de publicación) y asignarlo a una nueva columna, redondeado al centenar más cercano
df_merged['pricePublished'] = round(df_merged['priceSaleEbc'] * (1 + df_merged['factor final EBC']) / 100) * 100

In [ ]:
df_merged[df_merged['pricePublished'].isnull()][['llave_mma', 'MaxiPublica', 'priceSaleEbc','priceSaleEbc']]

In [ ]:
# Identificar y llenar NaN en 'pricePublished' con valores de 'MaxiPublica'
nan_price_published = df_merged['pricePublished'].isnull()
df_merged.loc[nan_price_published, 'pricePublished'] = df_merged.loc[nan_price_published, 'MaxiPublica']


In [ ]:

factorPricesSaleBusiness     = -0.08
factorPricesPurchaseBusiness = -0.13

# Calcular 'priceSaleBusiness' (precio de venta por negocios) y asignarlo a una nueva columna, redondeado al centenar más cercano
df_merged['priceSaleBusiness'] = round(df_merged['pricePublished'] * (1 + factorPricesSaleBusiness) / 100) * 100

# Calcular 'pricePurchaseBusiness' (precio de venta por negocios) y asignarlo a una nueva columna, redondeado al centenar más cercano
df_merged['pricePurchaseBusiness'] = round(df_merged['priceSaleBusiness'] * (1 + factorPricesPurchaseBusiness) / 100) * 100

# Mostrar el DataFrame combinado
display(df_merged[['Fecha', 'categoryId', 'brand', 'model', 'year', 'trim', 'MaxiPublica', 'llave_mm',
            'priceSaleEbc', 'priceSaleGam', 'Autos por marca modelo año', 'factor final EBC', 'pricePublished', 'priceSaleBusiness', 'pricePurchaseBusiness']])


df_merged.to_csv(f'{ruta_local_datos_temporal}/10_datos_historicos ajustados.csv', sep=',', index=False)
df_merged.to_excel(f'{ruta_local_datos_temporal}/10_datos_historicos ajustados.xlsx', index=False)

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


In [ ]:
# Obtiene los precios del mes mas reciente

# Filtrar las filas donde 'Fecha' sea igual a '2023-11-01'
df_mes_actual = df_merged[df_merged['Fecha'] == '2023-11-01']

#df_mes_actual.to_csv(f'{ruta_local_datos_temporal}/10_datos mes actual.csv', sep=',', index=False)
#df_mes_actual.to_excel(f'{ruta_local_datos_temporal}/10_datos mes actual.xlsx', index=False)

# Mostrar el DataFrame filtrado
display(df_mes_actual.head())

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


## Obtiene los precios del mes mas reciente

## Obtiene factores de direrencia entre los libros y el nuevo precio de venta

In [ ]:
#df_mes_actual.columns


# Hacer una copia del DataFrame para evitar las advertencias
df_mes_actual = df_mes_actual.copy()

# Función para calcular la diferencia porcentual para 'MP_vs_EBC'
def calcular_diferencia(row):
    if row['priceSaleBusiness'] != '' and row['priceSaleEbc'] != '':
        return (float(row['priceSaleBusiness']) - float(row['priceSaleEbc'])) / float(row['priceSaleBusiness'])
    else:
        return ''

# Aplicar la función a lo largo de las filas del DataFrame para 'MP_vs_EBC'
df_mes_actual['MPs_vs_EBC'] = df_mes_actual.apply(calcular_diferencia, axis=1)

# Función para calcular la diferencia porcentual para 'MP_vs_ATM'
def calcular_diferencia_atm(row):
    if row['priceSaleBusiness'] != '' and row['priceSaleGam'] != '':
        return (float(row['priceSaleBusiness']) - float(row['priceSaleGam'])) / float(row['priceSaleBusiness'])
    else:
        return ''

# Aplicar la función a lo largo de las filas del DataFrame para 'MP_vs_ATM'
df_mes_actual['MPs_vs_ATM'] = df_mes_actual.apply(calcular_diferencia_atm, axis=1)

# Mostrar el DataFrame con las columnas 'MP_vs_EBC' y 'MP_vs_ATM'
#display(df_mes_actual[['MPs_vs_EBC', 'MPs_vs_ATM']])
display(df_mes_actual.head())

df_mes_actual.to_csv(f'{ruta_local_datos_temporal}/10_datos mes actual.csv', sep=',', index=False)
df_mes_actual.to_excel(f'{ruta_local_datos_temporal}/10_datos mes actual.xlsx', index=False)

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


In [ ]:


# Calculando los percentiles y límites para MPs_vs_EBC y MPs_vs_ATM
Q0_EBC = df_mes_actual['MPs_vs_EBC'].quantile(0)
Q1_EBC = df_mes_actual['MPs_vs_EBC'].quantile(0.25)
Q2_EBC = df_mes_actual['MPs_vs_EBC'].quantile(0.5)
Q3_EBC = df_mes_actual['MPs_vs_EBC'].quantile(0.75)
Q4_EBC = df_mes_actual['MPs_vs_EBC'].quantile(1)
IQR_EBC = Q3_EBC - Q1_EBC
limite_inferior_EBC = Q1_EBC - 1.5 * IQR_EBC
limite_superior_EBC = Q3_EBC + 1.5 * IQR_EBC

Q0_ATM = df_mes_actual['MPs_vs_ATM'].quantile(0)
Q1_ATM = df_mes_actual['MPs_vs_ATM'].quantile(0.25)
Q2_ATM = df_mes_actual['MPs_vs_ATM'].quantile(0.5)
Q3_ATM = df_mes_actual['MPs_vs_ATM'].quantile(0.75)
Q4_ATM = df_mes_actual['MPs_vs_ATM'].quantile(1)
IQR_ATM = Q3_ATM - Q1_ATM
limite_inferior_ATM = Q1_ATM - 1.5 * IQR_ATM
limite_superior_ATM = Q3_ATM + 1.5 * IQR_ATM

# Creando un DataFrame con los resultados
data = {
    'MPs_vs_EBC': [Q0_EBC, limite_inferior_EBC, Q1_EBC, Q2_EBC, Q3_EBC, limite_superior_EBC, Q4_EBC],
    'MPs_vs_ATM': [Q0_ATM, limite_inferior_ATM, Q1_ATM, Q2_ATM, Q3_ATM, limite_superior_ATM, Q4_ATM]
}

index = ['Q0', 'lower_bound', 'Q1', 'Q2', 'Q3', 'upper_bound', 'Q4']

resumen_cuantiles = pd.DataFrame(data, index=index)
display(resumen_cuantiles)
resumen_cuantiles.to_csv(f'{ruta_local_datos_temporal}/11_resumen_cuantiles_vs_libros.csv', sep=',')


# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


In [ ]:
## gráficas EBC y ATM

In [ ]:
import matplotlib.pyplot as plt

# Graficar histograma para la columna MPs_vs_EBC
plt.figure(figsize=(6, 1.5))
plt.hist(df_mes_actual['MPs_vs_EBC'], bins=200, color='blue', alpha=0.7)
plt.xlabel('MPs_vs_EBC')
plt.ylabel('Frecuencia')
plt.title('Histograma de MPs_vs_EBC')
plt.grid(True)
plt.show()

# Graficar histograma para la columna MPs_vs_ATM
plt.figure(figsize=(6, 1.5))
plt.hist(df_mes_actual['MPs_vs_ATM'], bins=200, color='green', alpha=0.7)
plt.xlabel('MPs_vs_ATM')
plt.ylabel('Frecuencia')
plt.title('Histograma de MPs_vs_ATM')
plt.grid(True)
plt.show()

# Graficar histograma combinado para ambas columnas
plt.figure(figsize=(6, 1.5))
plt.hist(df_mes_actual['MPs_vs_EBC'], bins=200, color='blue', alpha=0.5, label='MPs_vs_EBC')
plt.hist(df_mes_actual['MPs_vs_ATM'], bins=200, color='green', alpha=0.5, label='MPs_vs_ATM')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')
plt.title('Histograma de MPs_vs_EBC y MPs_vs_ATM')
plt.legend()
plt.grid(True)
plt.show()

# Presenta la hora actual en UTC
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))


In [ ]:
# Filtrar los datos dentro de los límites lower_bound y upper_bound
filtered_data_EBC = df_mes_actual[(df_mes_actual['MPs_vs_EBC'] >= limite_inferior_EBC) & (df_mes_actual['MPs_vs_EBC'] <= limite_superior_EBC)]['MPs_vs_EBC']
filtered_data_ATM = df_mes_actual[(df_mes_actual['MPs_vs_ATM'] >= limite_inferior_ATM) & (df_mes_actual['MPs_vs_ATM'] <= limite_superior_ATM)]['MPs_vs_ATM']

# Graficar histograma para la columna MPs_vs_EBC filtrando los datos
plt.figure(figsize=(8, 3))
plt.hist(filtered_data_EBC, bins=200, color='blue', alpha=0.7)
plt.xlabel('MPs_vs_EBC')
plt.ylabel('Frecuencia')
plt.title('Histograma de MPs_vs_EBC (sin valores fuera de límites)')
plt.grid(True)
plt.show()

# Graficar histograma para la columna MPs_vs_ATM filtrando los datos
plt.figure(figsize=(8, 3))
plt.hist(filtered_data_ATM, bins=200, color='green', alpha=0.7)
plt.xlabel('MPs_vs_ATM')
plt.ylabel('Frecuencia')
plt.title('Histograma de MPs_vs_ATM (sin valores fuera de límites)')
plt.grid(True)
plt.show()

# Graficar histograma combinado para ambas columnas filtrando los datos
plt.figure(figsize=(8, 3))
plt.hist(filtered_data_EBC, bins=200, color='blue', alpha=0.5, label='MPs_vs_EBC')
plt.hist(filtered_data_ATM, bins=200, color='green', alpha=0.5, label='MPs_vs_ATM')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')
plt.title('Histograma de MPs_vs_EBC y MPs_vs_ATM (sin valores fuera de límites)')
plt.legend()
plt.grid(True)
plt.show()

### Gráfica de boxplot para EBC y ATM

In [ ]:
# Calcular cuartiles para 'MPs_vs_EBC' y 'MPs_vs_ATM'
Q1_EBC = df_mes_actual['MPs_vs_EBC'].quantile(0.25)
Q3_EBC = df_mes_actual['MPs_vs_EBC'].quantile(0.75)
IQR_EBC = Q3_EBC - Q1_EBC
limite_inferior_EBC = Q1_EBC - 1.5 * IQR_EBC
limite_superior_EBC = Q3_EBC + 1.5 * IQR_EBC

Q1_ATM = df_mes_actual['MPs_vs_ATM'].quantile(0.25)
Q3_ATM = df_mes_actual['MPs_vs_ATM'].quantile(0.75)
IQR_ATM = Q3_ATM - Q1_ATM
limite_inferior_ATM = Q1_ATM - 1.5 * IQR_ATM
limite_superior_ATM = Q3_ATM + 1.5 * IQR_ATM

# Filtrar los datos dentro de los límites
filtered_data_EBC = df_mes_actual[(df_mes_actual['MPs_vs_EBC'] >= limite_inferior_EBC) & (df_mes_actual['MPs_vs_EBC'] <= limite_superior_EBC)]['MPs_vs_EBC']
filtered_data_ATM = df_mes_actual[(df_mes_actual['MPs_vs_ATM'] >= limite_inferior_ATM) & (df_mes_actual['MPs_vs_ATM'] <= limite_superior_ATM)]['MPs_vs_ATM']

# Graficar el diagrama de cajas y bigotes con datos filtrados
plt.figure(figsize=(8, 6))
plt.boxplot([filtered_data_EBC, filtered_data_ATM], patch_artist=True)
plt.xlabel('Columnas')
plt.ylabel('Valores')
plt.title('Diagrama de Cajas y Bigotes para MPs_vs_EBC y MPs_vs_ATM (sin valores extremos)')
plt.xticks([1, 2], ['MPs_vs_EBC', 'MPs_vs_ATM'])
plt.grid(True)
plt.show()


# Copiar archivos a drive

#### Copia todos los archivos generados a Drive

In [ ]:
import shutil
import os

# Rutas de los directorios de origen y destino
ruta_origen = 'datos/tmp'
ruta_destino = '/content/drive/My Drive/Colab_Notebooks/precios/tmp/'

# Verificar si la ruta de destino existe, si no, crearla
if not os.path.exists(ruta_destino):
    os.makedirs(ruta_destino)

# Copiar todos los archivos de la ruta de origen a la ruta de destino
for archivo in os.listdir(ruta_origen):
    if not os.path.isdir(archivo):
        print(archivo)
        ruta_archivo_origen = os.path.join(ruta_origen, archivo)
        ruta_archivo_destino = os.path.join(ruta_destino, archivo)
        shutil.copy(ruta_archivo_origen, ruta_archivo_destino)


# Presenta la hora actual en UTC
print()
print("Terminé:", datetime.utcnow().astimezone(huso_horario_mexico).strftime('%Y-%m-%d %H:%M:%S %Z'))
